# Swarm analysis workflow (using new package structure)

面向 field-based-model 的简洁版流程示例，演示如何用新的命名空间导入和调用主要步骤。

In [5]:
# 环境 & 包导入
import numpy as np
import pandas as pd
import swarm
from swarm import data, preprocess, coarse, fields, observables, models, pde

## Step 0. 数据读取与预处理
- 读取原始 txt
- 可选平滑
- 补全速度/加速度/jerk

In [7]:
data_folder = "D:/3Ddataset/"  # TODO: 修改为实际路径
dfs = data.read_swarm_batch(data_folder, start=1, end=1)
df_raw = data.merge_dict_of_dfs(dfs)

# 可选：坐标平滑
#df_raw = data.apply_smoothing(df_raw)

# 标准预处理（中心化 + 速度/加速度/jerk）
df = preprocess.preprocess_full(df_raw)
df.head()

,id,x,z,y,t,vx,vz,vy,ax,az,ay,source,speed,accel,jx,jy,jz,jerk
0,1,-118.377364,-125.933758,108.973991,0.07,NaN,NaN,NaN,NaN,NaN,NaN,df1,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-119.172726,-124.756188,108.828342,0.08,-0.795362,1.177569,-0.145649,NaN,NaN,NaN,df1,1.428455,NaN,NaN,NaN,NaN,NaN
2,1,-112.160236,-123.504696,110.274999,0.09,7.012490,1.251492,1.446657,7.807852,0.073923,1.592306,df1,7.268704,7.968905,NaN,NaN,NaN,NaN
3,1,-105.905644,-128.482514,114.061838,0.10,6.254592,-4.977818,3.786839,-0.757898,-6.229310,2.340182,df1,8.845267,6.697400,NaN,NaN,NaN,NaN
4,2,-275.197364,260.906242,-199.396009,0.07,NaN,NaN,NaN,NaN,NaN,NaN,df1,NaN,NaN,NaN,NaN,NaN,NaN


## Step 1. 粗粒化字段 (histogram / Gaussian)
- 构建网格边界
- 生成密度/速度/加速度/jerk 场
- 也可用高斯核版本 `GaussianCoarseGrainer`

In [8]:
# 选一帧示例
t0 = np.sort(df["t"].unique())[0]
df0 = df[df["t"] == t0]

x_edges, y_edges, z_edges = coarse.make_grid_from_df(df0, grid_size=20.0, padding=5.0)
rho, (vx_field, vy_field, vz_field), edges = coarse.coarse_grain_velocity_frame(df0, x_edges, y_edges, z_edges)

# 高斯核示例
gcg = coarse.GaussianCoarseGrainer(grid_size=10.0, padding=5.0)
_ = gcg.build_grid(df0)
rho_g = gcg.coarse_density_sigma(df0, sigma=10.0)
vx_g, vy_g, vz_g = gcg.coarse_velocity_sigma(df0, sigma=10.0)

Grid size (Nx, Ny, Nz): 33 26 28
Grid: 66 51 55


## Step 2. 场操作与谱量
- 梯度/散度/旋度/拉普拉斯
- 速度相关函数 C(r) 与结构因子

In [9]:
xc = 0.5 * (x_edges[:-1] + x_edges[1:])
yc = 0.5 * (y_edges[:-1] + y_edges[1:])
zc = 0.5 * (z_edges[:-1] + z_edges[1:])

wx, wy, wz = fields.curl(vx_field, vy_field, vz_field, xc, yc, zc)

# 若 top-hat 粗粒化后有效体素过少，回退到高斯粗粒化结果
mask_valid = (~np.isnan(vx_field)) & (~np.isnan(vy_field)) & (~np.isnan(vz_field))
vx_use, vy_use, vz_use = vx_field, vy_field, vz_field
if mask_valid.sum() < 10:
    print("Top-hat grid too sparse, fallback to Gaussian coarse-grain for correlation.")
    vx_use, vy_use, vz_use = vx_g, vy_g, vz_g

mask_valid = (~np.isnan(vx_use)) & (~np.isnan(vy_use)) & (~np.isnan(vz_use))
if mask_valid.sum() < 10:
    print("Still too few voxels; consider larger grid_size/padding or denser data.")
    r_vals, C_v = np.array([]), np.array([])
else:
    r_vals, C_v = fields.compute_velocity_correlation(vx_use, vy_use, vz_use, xc, yc, zc, nbins=20)

k_vals, S_rho = fields.compute_structure_factor_3D(rho)
r_vals, (C_v[:5] if len(C_v) else C_v), k_vals[:5], S_rho[:5]

Top-hat grid too sparse, fallback to Gaussian coarse-grain for correlation.
Still too few voxels; consider larger grid_size/padding or denser data.


(array([], dtype=float64),
 array([], dtype=float64),
 array([0.01099188, 0.03297565, 0.05495942, 0.07694319, 0.09892695]),
 array([49.44324014, 51.08584572, 74.45172965, 79.44049045, 89.32127429]))

## Step 3. 粒子态观测量
- pair correlation g(r)
- S(q), S_v(q)
- 局部对齐

In [ ]:
r_centers, g_r = observables.compute_pair_correlation(df, dr=5.0, min_particles=30)
q_vals, S_q = observables.compute_structure_factor(df, q_max=0.5, dq=0.02)
q_vals_v, S_v = observables.compute_velocity_structure_factor(df, q_max=0.5, dq=0.02)
align_score = observables.analyze_local_alignment(df, radius=20.0)
g_r[:5], S_q[:5], S_v[:5], align_score

## Step 4. 建模 (MEM / RG / PDE)
- MEM：用单位速度向量与邻域列表拟合 J
- RG：`models.power_law_fit` 等工具
- PDE：基于粒子 ID 的拉格朗日拟合

In [10]:
# MEM 示例（伪代码，需要邻居列表 neighbor_list）
s = df0[["vx","vy","vz"]].values
s = s / np.linalg.norm(s, axis=1, keepdims=True)
neighbor_list = []  # TODO: 构造邻接关系
# C_data = models.compute_C_data(s, neighbor_list)
# m = models.compute_polarization(s)
# 使用 F(J, C_data, m)=0 求解 J

# PDE 拟合示例（加速度 PDE）
try:
    res_acc = pde.fit_acc_pde_lagrangian(df, t0=None, dt=None, length_scale=15.0, k_neighbors=10)
    res_acc
except Exception as e:
    print("PDE fit skipped/failed:", e)

[Lagrangian PDE fit] t0 = 55.01, t1 = 55.02, dt = 0.010000000000005116
  matched particles = 97
  used particles after masking: 82

[Fitted Lagrangian PDE parameters]
  D_hat     = 1740.612053
  gamma_hat = 153.274819
  residual ratio (Lagrangian) = 0.1230
